# Parcial 09 de septiembre de 2019

## Nombre:

Se han tomado muestras de granos de café de 99 fincas cafeteras diferentes. De cada muestra se tomó una medida del PH, la humedad, los grados Brits, los sabores y los olores, y se le ha pedido a 4 expertos que, de acuerdo a estas medidas, determinen si el café es de calidad o no. Cada experto asigna un 1 si le parece de calidad y un 0 si le parece que no es de calidad. Con esta información se le pide que desarrolle un sistema que haga las veces de experto, de tal forma, a partir de las mismas 4 característicos de una muestra de café, sea capaz de determinar si es de calidad o no. Para la construcción del sistema se tienen los siguientes insumos:

1. Los datos de las muestras de café se encuentran en el archivo $\verb|Cafe.txt|$. La matriz de observaciones es de tamaño 100x5
2. Las etiquetas asignadas por los expertos se encuentran en el archivo $\verb|Expertos.txt|$, estas tienen tamaño 100x4 (1 columna por cada uno de los expertos). Tenga en cuenta que en total cada observación tiene 4 etiquetas, una por experto. La forma en que usted decida trabajar con ellas hará parte del sistema propuesto.
3. En el archivo $\verb|Test.txt|$ se encuentra la información de 19 muestras de café, de las que se requiere conocer la calidad a través de su sistema. 

El sistema experto debe cumplir con las siguientes condiciones:

1. Tener una etapa de preprocesamiento de las observaciones.
2. Se debe hacer una partición de entrenamiento y validación para probar el sistema desarrollado.
3. Tener al menos dos métodos para la clasificación  de la información, uno de los cuales debe ser árboles de decisión.
4. Después de tener el sistema entrenado y validado se debe dar la información  de calidad para las 19 muestras de café de las que no se tiene conocimiento.
5. Cualquier tipo de información adicional que crea requerir la debe suponer. Cada supuesto debe estar perfectamente justificado de forma explícita en formato de comentarios dentro del código


In [111]:
#espacio para resolver el parcial

import matplotlib.pyplot as plt 
from sklearn.metrics import classification_report  #esta librería nos ayudará a tener a la salida diferentes métricas, no solo el acierto de clasificación
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier #librería para el árbol de decisión
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors, datasets
from sklearn.naive_bayes import GaussianNB 
import numpy as np
import statistics

#LEO LOS DATOS

input_cafe = 'Cafe.txt'
datos_cafe = np.loadtxt(input_cafe,delimiter =',')
cafe = datos_cafe[:,:]

input_expertos = 'Expertos.txt'
datos_expertos = np.loadtxt(input_expertos,delimiter =',')

input_test = 'Test.txt'
datos_test = np.loadtxt(input_test,delimiter =',')


valoracion = np.zeros(datos_expertos.shape[0]).astype(np.int64)
uno = 0
cero = 0
# en este for lo que voy a hacer es que de acuerdo a todas las opiniones de los expertos digo si la muestra es buena 
#por ejempplo si en la primera muestra la mayoria dice que es buena (1) entonces la tomo como uno.
for i in range(datos_expertos.shape[0]):
    for j in range(datos_expertos.shape[1]):
        if datos_expertos[i,j] == 0:
            cero = cero + 1
        if datos_expertos[i,j] == 1:
            uno = uno + 1
        if cero > uno:
            valoracion[i] = 0
        if uno > cero:
            valoracion[i] = 1
        if uno == cero:#este if es para ver cuando la mitad de opniones es buena y la otra mala, entonces escojo la opinion buena
            valoracion[i] = 1
    
    uno = 0
    cero = 0
    
# print(valoracion)
#particiono los datos
X_train = cafe
y_train = valoracion

X_test = datos_test

# X_train, X_test, y_train, y_test = train_test_split(cafe,valoracion, test_size = 0.2, random_state = 5)

#etapa de preprocesamiento
# Remoción
def remocion(X_train, X_test):
    media = X_train.mean(axis = 0)
    desviacion = X_train.std(axis = 0)
    X_train_prepro = (X_train - media)/desviacion
    X_test_prepro = (X_test - media)/desviacion
    return X_train_prepro, X_test_prepro


X_train_preproR, X_test_preproR = remocion(X_train, X_test)


#usaré para kNN
NN = 59 #definimos le número de vecinos (en lo posible elegir números impares)
clasificadorkNN = neighbors.KNeighborsClassifier(NN, weights = 'distance') #instanciamos el clasificador

#entreno el clasificador
clasificadorkNN.fit(X_train_preproR,y_train)
# se predice, de acuerdo a este sistema una valoracion para las 19 muestras

#profe soy 20 muestras para el test
#hay algo importamte  aquí y es que a la hora de ver que muestra es buena o mala, de acuerdo a los expertos, y en una de esas opiniones
#la mitad dice buena y la otra mitad mala, dependiendo de lo que escoja, es decir, la pongo mala o buena, cambia mucho el clasificador, yo la coloco buena

valoracion_pred_kNN = clasificadorkNN.predict(X_test_preproR)
print( 'kNN =' ,valoracion_pred_kNN)

#pruebo otro clasificador
clasificadorNB = GaussianNB()
clasificadorNB.fit(X_train_preproR,y_train)
valoracion_pred_NB = clasificadorNB.predict(X_test_preproR)
print('gausiana = ',valoracion_pred_NB)



#procedo a utilizar arboles

parametros = {'n_estimators':150,'random_state':0,'max_depth':4}
clasificadorAR = ExtraTreesClassifier(**parametros)
clasificadorAR.fit(X_train_preproR, y_train)
valoracion_pred_AR = clasificadorAR.predict(X_test_preproR)
print('arboles = ',valoracion_pred_AR)








kNN = [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
gausiana =  [1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0]
arboles =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
